## Bigkinds 크롤링


## 필요한 라이브러리 설치
* 아나콘다 사용시 다음의 프롬프트 창을 열어 conda 명령어로 설치합니다.
* pip 사용시 아래에 있는 명령어를 터미널로 설치합니다.
<img src="https://i.imgur.com/Sar4gdw.jpg">

### Selenium
* `conda install -c anaconda selenium`
* [Selenium :: Anaconda Cloud](https://anaconda.org/anaconda/selenium)

* pip 사용시 : `pip install selenium`

### BeautifulSoup
* `conda install -c anaconda beautifulsoup4`
* [Beautifulsoup4 :: Anaconda Cloud](https://anaconda.org/anaconda/beautifulsoup4)

* pip 사용시 : `pip install beautifulsoup4`

### tqdm
* `conda install -c conda-forge tqdm`
* [tqdm/tqdm: A Fast, Extensible Progress Bar for Python and CLI](https://github.com/tqdm/tqdm)
* `pip install tqdm`

In [1]:
def get_login(id, pwd):     
    driver.find_element_by_xpath('//*[@id="app-top-navbar"]/ul/li[1]/a').click()
    time.sleep(1)

    driver.find_element_by_xpath('//*[@id="login-user-id"]').send_keys(id)
    driver.find_element_by_xpath('//*[@id="login-user-password"]').send_keys(pwd)
    driver.find_element_by_xpath('//*[@id="login-btn"]').click()
    time.sleep(1)

def set_date(startdate, enddate):
    driver.find_element_by_xpath('//*[@id="date-filter-btn"]').click()
    driver.find_element_by_xpath('//*[@id="search-begin-date"]').send_keys(Keys.CONTROL + "a")
    driver.find_element_by_xpath('//*[@id="search-begin-date"]').send_keys(Keys.DELETE)
    driver.find_element_by_xpath('//*[@id="search-begin-date"]').send_keys(startdate)
    time.sleep(2)
    
    driver.find_element_by_xpath('//*[@id="search-end-date"]').send_keys(Keys.CONTROL + "a")
    driver.find_element_by_xpath('//*[@id="search-end-date"]').send_keys(Keys.DELETE)    
    driver.find_element_by_xpath('//*[@id="search-end-date"]').send_keys(enddate)
    time.sleep(2)
    
    driver.find_element_by_xpath('//*[@id="date-confirm-btn"]').click()
    time.sleep(2)

    
def get_postContents(keyword, num_news, interval):

#     keyword_url = base_url + keyword
    
    titles = []
    contents = []
    dates = []
    soup_titles = []
    soup_contents = []
    soup_dates = []
    post_df = pd.DataFrame(columns=('title', 'content', 'date'))
    
    # 메인페이지로 이동한다
    driver.get('https://www.bigkinds.or.kr/')
    
    # 조회 기간을 입력한다
    startdate='2019-07-01'
    enddate='2020-06-30'
    set_date(startdate, enddate)
 
#     print("keyword 입력하고 검색버튼을 누른다")
    driver.find_element_by_xpath('//*[@id="total-search-key"]').send_keys(keyword)
    driver.find_element_by_xpath('//*[@id="news-search-form"]/div/div/div/div[1]/span/button').click()
    time.sleep(3)  
        
    soup = bs(driver.page_source, 'html.parser')
    total_cnt_str = soup.find("span", id="total-news-cnt").get_text()
    total_cnt = int(total_cnt_str.replace(",", ""))

#     print("검색된 전체 뉴스를 스크랩 한다")
    pages = (total_cnt // 10) + 1
    for page in trange(0, pages):
        
#         print("page 를 넘길 때 인덱스를 사용한다. 여기서는 7 배수를 사용한다.")
        index = (page % 7) + 4

        try:
#             print("필요한 정보만 추려낸다")
            soup = bs(driver.page_source, 'html.parser')
#             news = soup.find("div", id="news-results")
            soup_titles = soup.find_all('h4', attrs={"class": "news-item__title news-detail"})
            soup_contents = soup.find_all('div', attrs={"class": "news-item__content news-detail"})
            soup_dates = soup.find_all('span', attrs={"class": "news-item__date"})

#             print("해당 페이지로 이동한다")
            xpath = f'//*[@id="news-results-pagination"]/ul/li[{index}]/a'
#             print (xpath)
            driver.find_element_by_xpath(xpath).click()
            time.sleep(interval)   

#             print("추려낸 내용을 깔끔하게 '개행문자와 공백 제거' 정리한다")
            for title, content, date in zip(soup_titles, soup_contents, soup_dates):
                title = title.get_text().replace("\n","").strip()
                titles.append(title)
                content = content.get_text().replace("\n","").strip()
                contents.append(content)
                date = date.get_text().replace("\n","").strip()
                dates.append(date)
        except:
            pass
                     
#         print(titles)
            
#     print("추려낸 내용을 dataframe 에 저장 한다")
    post_df = pd.DataFrame({ "title": titles, "content": contents, "date": dates })
    
#     print("dataframe 를 반환한다")
    return post_df

def letsfind(keywords):

    # base_url = f"https://m.kin.naver.com/mobile/search/searchList.nhn?cs=utf8&query="
    num_news = 50000
    interval = 3

    try:
        post_df = get_postContents(keyword, num_news, interval)
        filename = "./scraps/" + "bigkinds_" + keyword.replace(" ","") + ".csv"
        post_df.to_csv(filename, date_format='%Y%m%d', encoding='utf-8-sig') 
    except:
        pass

##############################################################################################
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져온다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱한다.
from bs4 import BeautifulSoup as bs
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러온다.
import pandas as pd
from pandas import read_excel
from selenium import webdriver
# from seleniumrequests import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import datetime as dt
import time
from tqdm import trange
from tqdm import tqdm

##############################################################################################

# 로컬 PC
options = webdriver.ChromeOptions()
options.headless = False
options.add_argument("window-size=1920x1080") # 가상화면 크기
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36")

# # Docker
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# options = webdriver.ChromeOptions()
# options.headless = True
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# options.add_argument("window-size=1920x1080") # 가상화면 크기
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36")

# 크롬 드라이버 로드한다.
driver = webdriver.Chrome(options=options)
# 암묵적으로 웹 자원 로드를 위해 3초까지 기다려 준다.
driver.implicitly_wait(5)
# 화면을 최대로 키운다
driver.maximize_window()
# 메인페이지로 이동한다
driver.get('https://www.bigkinds.or.kr/')

# 로그인 한다
id = 'inhwan.jung@gmail.com'
pwd = '2020@alpha'
get_login(id, pwd)
    
# 조회할 키워드를 설정한다.
##################### Keyword 루프 시작 ##################### 
keywords = ['외식', '식사', '메뉴', '식당', '레스토랑', '가성비', '트렌드', '배달', '맛집', '비대면']

for keyword in keywords:
    letsfind(keyword)
    

100%|███████████████████████████████████████████| 3702/3702 [4:19:57<00:00,  4.21s/it]
